In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
! pip install accelerate peft bitsandbytes git+https://github.com/huggingface/transformers trl py7zr auto-gptq optimum

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-1o5hrcf6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-1o5hrcf6
  Resolved https://github.com/huggingface/transformers to commit b5a6d6eeab2524546776057b40774835c45c4971
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.

In [3]:
import pandas as pd
df=pd.read_csv(r"/content/medquad.csv")

In [4]:
df=df[['question','answer']]

In [5]:
df.head()

,question,answer
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea..."
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ..."
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...


In [6]:
df.dropna(inplace=True)

In [7]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os

In [8]:
df["text"] = df.apply(lambda x: "### Human: " + str(x["question"]) + " ### Assistant: " + str(x["answer"]), axis=1)

In [9]:
df.to_csv("/content/med.csv")

In [10]:
data = Dataset.from_pandas(df)


In [11]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [12]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)
model = AutoModelForCausalLM.from_pretrained(
                            "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
                            quantization_config=quantization_config_loading,
                            device_map="auto"
                        )


Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [13]:
model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


peft_config = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)

In [14]:
training_arguments = TrainingArguments(
        output_dir="mistral-finetuned-alpaca",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=50,
        num_train_epochs=1,
        max_steps=300,
        fp16=True,
        push_to_hub=True
)


In [15]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=512
)


trainer.train()

# time taken 1h 3m

Map:   0%|          | 0/16407 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
50,1.336800
100,1.129200
150,1.050400
200,1.077300
250,1.037100
300,1.018100


TrainOutput(global_step=300, training_loss=1.1081423187255859, metrics={'train_runtime': 3796.0917, 'train_samples_per_second': 0.632, 'train_steps_per_second': 0.079, 'total_flos': 959575574052864.0, 'train_loss': 1.1081423187255859, 'epoch': 0.15})

In [20]:
! cp -r /content/mistral-finetuned-alpaca /content/drive/MyDrive/

cp: cannot create directory '/content/drive/MyDrive/': No such file or directory


In [21]:
!zip -r mistral-finetuned-alpaca.zip mistral-finetuned-alpaca


updating: mistral-finetuned-alpaca/ (stored 0%)
updating: mistral-finetuned-alpaca/checkpoint-300/ (stored 0%)
updating: mistral-finetuned-alpaca/checkpoint-300/scheduler.pt (deflated 57%)
updating: mistral-finetuned-alpaca/checkpoint-300/special_tokens_map.json (deflated 72%)
updating: mistral-finetuned-alpaca/checkpoint-300/training_args.bin (deflated 51%)
updating: mistral-finetuned-alpaca/checkpoint-300/tokenizer.json (deflated 74%)
updating: mistral-finetuned-alpaca/checkpoint-300/optimizer.pt (deflated 8%)
updating: mistral-finetuned-alpaca/checkpoint-300/README.md (deflated 66%)
updating: mistral-finetuned-alpaca/checkpoint-300/adapter_model.safetensors (deflated 8%)
updating: mistral-finetuned-alpaca/checkpoint-300/tokenizer.model (deflated 55%)
updating: mistral-finetuned-alpaca/checkpoint-300/trainer_state.json (deflated 65%)
updating: mistral-finetuned-alpaca/checkpoint-300/rng_state.pth (deflated 25%)
updating: mistral-finetuned-alpaca/checkpoint-300/adapter_config.json (de

In [22]:
from google.colab import files
files.download('mistral-finetuned-alpaca.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# Assuming you've already loaded or fine-tuned your model
# and have it ready to use, similar to the previous setup

# Load tokenizer and model (example path, replace with your actual model path if different)
tokenizer = AutoTokenizer.from_pretrained("/content/mistral-finetuned-alpaca")
model = AutoModelForCausalLM.from_pretrained("/content/mistral-finetuned-alpaca",
                                             low_cpu_mem_usage=True,
                                             return_dict=True,
                                             torch_dtype=torch.float16,
                                             device_map="cuda")

# Now, accept dynamic user input
user_question = input("Please enter your question: ")

# Formatting the input text for the model
input_text = f"""###Human:"{user_question}" ? ###Assistant: """
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

# Set your generation configuration as per your requirement
generation_config = GenerationConfig(
    do_sample=True,
    top_k=50,
    temperature=0.7,
    max_new_tokens=150,
    pad_token_id=tokenizer.eos_token_id,
)

# Generate response
outputs = model.generate(**inputs, **vars(generation_config))

# Decode the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Post-process to trim at the last full stop for a complete sentence
last_full_stop_index = generated_text.rfind('.')
if last_full_stop_index != -1:
    generated_text = generated_text[:last_full_stop_index + 1]

# Output the generated text
print("Generated Text:")
print("-" * 20)
print(generated_text)

Please enter your question: What is the outlook for Atypical Chronic Myelogenous Leukemia ?


Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Text:
--------------------
###Human:"What is the outlook for Atypical Chronic Myelogenous Leukemia ?" ? ###Assistant:  Atypical chronic myelogenous leukemia (CML) is a rare type of cancer that affects the bone marrow and blood. The bone marrow produces many different cells that help to fight infections and control blood clotting. In CML, the bone marrow and blood contain too many white blood cells. The excess white blood cells are called leukemia cells. The leukemia cells do not fight infection and have abnormal functions that can cause bleeding. Researchers are working to learn more about the causes of CML and to develop ways to prevent, diagnose, and treat this disease.


In [23]:
# from peft import AutoPeftModelForCausalLM
# from transformers import GenerationConfig
# from transformers import AutoTokenizer
# import torch
# tokenizer = AutoTokenizer.from_pretrained("/content/mistral-finetuned-alpaca")

In [36]:
# inputs = tokenizer("""###Human:"What is (are) Glaucoma ?" ? ###Assistant: """, return_tensors="pt").to("cuda")

In [37]:
# model = AutoPeftModelForCausalLM.from_pretrained(
#     "/content/mistral-finetuned-alpaca",
#     low_cpu_mem_usage=True,
#     return_dict=True,
#     torch_dtype=torch.float16,
#     device_map="cuda")

In [38]:
# # Adjusted generation configuration
# generation_config = GenerationConfig(
#     do_sample=True,         # Enable sampling to generate diverse responses
#     top_k=50,               # Consider the top 50 tokens at each step for sampling
#     temperature=0.7,        # Adjust temperature to balance randomness and fidelity
#     max_new_tokens=150,     # Increase max_new_tokens to allow for longer responses
#     pad_token_id=tokenizer.eos_token_id,
#     eos_token_id=tokenizer.eos_token_id,  # Ensure the model respects the EOS token
# )

In [41]:
# import time

# # Start time
# start_time = time.time()

# # Generate outputs
# outputs = model.generate(**inputs, **vars(generation_config))

# # Decode the generated text
# generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# # Post-process to trim the output at the last full stop
# # Find the last occurrence of a full stop and keep the text up to that point
# last_full_stop_index = generated_text.rfind('.')
# if last_full_stop_index != -1:
#     generated_text = generated_text[:last_full_stop_index + 1]

# print("Generated Text:")
# print("-" * 20)
# print(generated_text)
# print("-" * 20)

# # Calculate and print the time taken
# elapsed_time = time.time() - start_time
# print(f"Time taken: {elapsed_time:.2f} seconds")


# # import time

# # # Start time
# # start_time = time.time()

# # # Generate outputs
# # #outputs = model.generate(**inputs, generation_config=generation_config)
# # outputs = model.generate(**inputs, **vars(generation_config))

# # # Decode and print the generated text
# # generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# # print("Generated Text:")
# # print("-" * 20)
# # print(generated_text)
# # print("-" * 20)

# # # Calculate and print the time taken
# # elapsed_time = time.time() - start_time
# # print(f"Time taken: {elapsed_time:.2f} seconds")


Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Text:
--------------------
###Human:"What is (are) Glaucoma ?" ? ###Assistant:  Glaucoma is a group of diseases that damage the eye's optic nerve and lead to progressive vision loss. The eye's optic nerve is made up of many small fibers that transmit visual information from the retina, the layer of cells at the back of the eye, to the brain. Glaucoma can cause the fibers of the optic nerve to break down or become damaged, leading to vision loss. People with glaucoma may experience vision loss that cannot be reversed. However, in some cases, vision loss may be slowed or prevented with treatment. The most common form of glaucoma is primary open-angle glaucoma.
--------------------
Time taken: 10.11 seconds


  Cloning https://github.com/huggingface/optimum.git to /tmp/pip-install-1olafiyw/optimum_4167f51172c94afe99465accadf61aed
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/optimum.git /tmp/pip-install-1olafiyw/optimum_4167f51172c94afe99465accadf61aed
  Resolved https://github.com/huggingface/optimum.git to commit 7e08a820b65a359a61444abe51df4eb96b26b2e3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.0 MB/s eta 0:00:00
  Created wheel for optimum: filename=optimum-1.18.0.dev0-py3-none-any.whl size=409947 sha256=7d993f68dd90d2a3608e2ea920085ee63645af07259f49496bfa17979a8f8ccc
  Stored in directory: /tmp/pip-ephem-wheel-cache-3bba2w2x/wheels/8f/fa/

In [ ]:
!unzip "/content/drive/MyDrive/Dataset/mistral-finetuned-alpaca.zip"

In [ ]:
pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/